In [1]:
import pandas as pd

Começaremos estudando machine learning através das árvores de decisão. São simples de entender e são também a base de outros modelos preditivos mais avançados. Vamos utilizar as árvores de decisão para nos ajudar a prever preços de casa.

O modelo aprende alguns padrões a partir de dados históricos e nos ajuda a fazer a previsão. Essa etapa onde o modelo aprende os padrões é chamado de *fitting* ou *training*, pois estamos de fato treinando o modelo para que aprenda sobre os dados. Os dados utilizados para isso são chamados de *training data*.

Primeiramente nossa árvore de decisão vai ter um caminho bem simples, ela divide as casas em apenas duas categorias. Se a casa tem mais ou menos que 2 quartos. Veja imagem abaixo:

![image](tree0.png)

Melhorando nossa árvore de decisão, agora ela considera mais variáveis e ficou um pouco mais complexa. Ela agora pesa mais fatores antes de decidir qual o valor de uma casa. Além da quantidade de quartos, ela considera também o tamanho da casa. Essa que tem mais divisões chamamos de árvores mais profundas.

![image](tree2.png)

Podemos prever o valor da casa percorrendo o caminho da árvore correspondente às características da casa. O ponto onde é feita a previsão é chamado de folha.

---

### Basic Data Exploration

Instruções sobre estatística básica e primeiras linhas de código com pandas. O curso dá uma visão básica de como é importante fazer um EDA para conhecer os dados. E já mostra a função ```df.describe()```. Fala um pouco de que são média, mediana e percentis, ajudando a interpretar o dataset dos preços das casas.

---

### Your First Machine Learning Model

Aqui ensina como selecionar colunas com o pandas. Tem a notação com ponto e tem a seleção através dos nomes das colunas dentro dos colchetes. Os nomes das colunas são chamadas de features.

Para um modelo de machine learning supervisionado precisamos separar a coluna de target, que é o que queremos prever, e as colunas de features, que são os preditores.

Abaixo por exemplo estaríamos atribuindo a variável y o nosso target do dataset de casas:

```y = df_house['price']```

E aqui as features:

```features = ['rooms', 'bathrooms', 'landsize', 'lat', 'long']```

Por convenção chamamos esses dados de X:

```X = df_house[features]```


#### Building Your Models

Usaremos o scikit-learn para criar nossos modelos. As etapas para a criação de um modelo são:

- Definir: qual modelo utilizar..será uma árvore de decisão, outro tipo...
- Fit: capture os padrões dos dados históricos
- Predict: tente fazer previsões usando novos dados
- Avaliar: determinar o quão preciso nosso modelo está performando

---

Aqui um exemplo de como usar o scikit-learn para criar uma árvore de decisão.

```python
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

# Fit model
melbourne_model.fit(X, y)
```

Primeiro instanciamos o modelo e atribuímos um random state. Depois fornecemos os valores X e y a esse modelo instanciado.

E para fazer as previsões basta fazer o seguinte:

```python

melbourne_model.predict(df)

```

Este df fornecido para o modelo prever são dados novos, mas com as mesmas features (colunas) dos dados de treinamento.

---



### Model Validation

Ok, construimos nosso modelo. Mas, será que ele é bom, como podemos avaliar isso.

Essa é uma etapa importante no ciclo de um projeto de data science.

Medimos a qualidade do nosso modelo através da precisão da previsão. Ou seja, o quão longe nossa previsão ficou do valor real? A métrica usada para isso é o MAE (mean absolute error), que é a média dos erros absolutos. Basicamente vemos a diferença para mais ou para menos de todas as casas e tiramos a média dessas diferenças. Precisamos dos valores absolutos para retirarmos o sinal de negativo e assim podermos calcular a média. A pergunta que essa métrica nos ajuda a responder é: "em média, quanto meu modelo está errando?"

Veja no exemplo abaixo como fica isso no código:



```python
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(df)

mean_absolute_error(y, predicted_home_prices)

# output: 434.71594577146544
```

>> Veja que o resultado é $ 434.71, ou seja, seu modelo está errando a previsão em relação ao valor real em menos de 500 dólares. É uma ótima previsão em se tratando de valores de casas e apartamentos. Mas saiba que essa precisão toda pode não ser muito boa, como veremos a seguir...

#### The Problem with In-Sample Scores

Perceba que utilizamos o mesmo conjunto de dados para treinar e avaliar o modelo. Isso não é boa coisa e já explico o porquê. Imagine que por algum motivo no seu conjunto de dados todas as casas com porta azul tinham os preços mais caros. Seu modelo vai aprender esse padrão. E quando vc tentar fazer previsões com novos conjuntos de dados ele vai acabar ficando enviesado, estimando erroneamente para mais o valor de todas casas com porta azul. Daí acontece que, se vc testá-lo usando o mesmo conjunto de treino então os resultados sairão ótimos já que o modelo já viu esse dado antes. Porém não veremos o mesmo desempenho na precisão quando trata-se de dados novos. Ou seja, não queremos que o modelo aprenda demais sobre os dados de treino, é importante que ele generalize.

A solução para isso é utilizar um mesmo conjunto de dados para treino e para teste. Ao criar o modelo, separamos uma parte dos dados para treino e outra para validação, assim podemos testar o modelo com dados reais e que ele não viu antes.

O scikit-learn tem um método para nos ajudar nessa divisão do dataset, é o método ```train_test_split```. Agora faremos essa separação no conjunto de dados e depois vamos novamente fazer o teste MAE para ver como o modelo se comporta. Vamos ver como fica no código:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error


# criando variáveis fake só para dar o exemplo
X = 'features'
y = 'target'

# dividimos o dataset em treino e teste
# isso para ambos features e target
train_X, test_x, train_y, test_y = train_test_split(X, y, random_state=42)

# agora definimos o modelo
melbourne_model = DecisionTreeRegressor()

# treinamos o modelo
melbourne_model.fit(train_X, train_y)

# agora vamos fazer umas previsões 
# e vamos fazer o teste de precisão
predicitions = melbourne_model.predict(test_x)

# testando com o método MAE
print(mean_absolute_error(test_y, predicitions))

>>output: 265806.91478373145

Aqui vimos como é importante separar o dataset em treino e teste com a ajuda do scikit-learn.

Para a maioria dos modelos esse passo a passo será o mesmo. Ou seja, separamos os dados em features e target, depois fazemos o split, instanciamos o modelo, treinamos esse modelo com o split de treino, fazemos as previsões com o split de x_test e depois validamos com o y_test.

Veja que agora a precisão ficou bem pior, de apenas $ 434 para $ 265.000! Perceba o perigo de confiar no modelo anterior, treinado e testado com o mesmo conjunto de dados.

Mas não se preocupe, existem várias maneiras de melhorarmos este modelo, tais como encontrar melhores features e/ou outros modelos. Veremos mais ao longo do curso.

---

### Underfitting and Overfitting

- Underfitting: quando o modelo falha em capturar padrões nos dados, levando a previsões menos precisas.

- Overfitting: quando o modelo aprende demais sobre os dados. Ele captura padrões muito específicos daquele dataset somente (lembre do caso das cores das portas). Dessa forma, ele também resulta em previsões menos precisas com novos datasets.

No caso das árvores de decisão mais profundas, quando existem mais folhas, os dados vão sendo divididos até chegar a decisão final. Com menos dados para tomar a decisão a previsão fica menos precisa. Funciona bem com os dados de treino, pois as características da casa sempre vão estar lá na árvore. Mas funciona mal com novos dados. Estes são casos de **overfitting**.

Já quando a árvore tem menos folhas, o modelo não divide bem os dados em características distintas. Menos divisões, então muito mais dados. Quando o modelo falha em capturar diferenças significativas e padrões nos dados então temos caso de **underfitting**.

Nós queremos encontrar um meio termo, nem modelo treinado demais, nem de menos.

Para procurar esse meio termo vamos testar profundidades diferentes para cada modelo. E no final vamos eleger o de melhor performance.



In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split


# criando uma função para depois fazer a iteração
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [ ]:
import pandas as pd
    
# carregando os dados
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# atribuindo o target e as features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

# fazendo o split em treino e teste
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

In [ ]:
# agora vamos comparar o valor de MAE para diferentes
# níveis de profundidade
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

output:

Max leaf nodes: 5  		     Mean Absolute Error:  347380\
Max leaf nodes: 50  		 Mean Absolute Error:  258171\
Max leaf nodes: 500  		 Mean Absolute Error:  243495\
Max leaf nodes: 5000  		 Mean Absolute Error:  254983

Com esse resultado vemos que a melhor opção é de 500 nós (folhas).

Agora que já fizemos as validações, encontramos os melhores parametros e já tomamos as decisões de modelagem, podemos dispensar a fase de validação e treinar o modelo com todos os dados disponíveis, assim melhoramos a precisão. 

In [ ]:
# vamos instanciar o modelo novamente com o parametro ideal encontrado
final_model = DecisionTreeRegressor(max_leaf_nodes=500, random_state=0)

# usaremos agora todo o dataset para treinar o modelo
final_model.fit(X, y)

Veja que conseguimos melhorar nosso modelo, mas ainda estamos usando árvores de decisão simples, que não são modelos tão sofisticados. Vamos estudar agora um modelo derivado das árvores de decisão, mas muito mais poderoso.

---

### Random Forests

Florestas Aleatórias (Random Forests) são mais poderosas pois são várias árvores de decisão para chegar a uma previsão. Ou seja, ele tira uma média das previsões de todas as árvores de decisão. Dessa forma, esse modelo tem uma precisão preditiva muito maior que uma simples árvore de decisão, além de funcionar muito bem com os parametros default.

A diferença no código é bem básica, na etapa de instanciar o modelo, basta instanciar o ```RandomForestRegressor```. O modo de fazer o fit e predições continuam os mesmos. Veja abaixo:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

>>Output: 191669.7536453626

Veja como já melhoramos bastante. Nem ajustamos nenhum parametro, usamos os padrões, e ainda assim o modelo já retornou um resultado melhor que o da árvore de decisão, que era de 250.000.

---

## Intermediate Machine Learning

### Missing Values

Este capítulo dá instruções sobre como lidar com missing values. Estes valores faltantes não podem ser ignorados. Ou pelo menos vc precisa estar ciente do porquê estes valores estão faltando. A forma que vc lida com eles pode afetar significativamente o desempenho do modelo.

Existem basicamente 3 approaches:

- 1. Dropar as colunas com missing values
    - Opção mais perigosa, tem que saber o que está fazendo..pois corremos o risco de perder features importantes. Imagine perder toda uma coluna com milhares de linhas por causa de uns poucos missing values! Não é a melhor das opções.
    
- 2. Imputação
    - Preencher os missing values com valores. Estes valores podem ser a média da sequencia, a moda...nem sempre vão refletir a realidade, mas influenciam positivamente na precisão dos modelos preditivos e trazem melhores resultados que a opção anterior.
- 3. Extensão da imputação
    - Faz o mesmo processo de imputação dito anteriormente. Mas neste caso inclui mais uma coluna na tabela, indicando a origem do valor da coluna de referência. Por exemplo, imputaríamos a coluna 'salario' com a média. Teria uma coluna adicional 'salario_missing' que indicaria com ```True``` ou ```False``` se o valor da coluna 'salario' foi imputado ou não.

O autor faz uma comparação do desempenho das 3 abordagens, e o melhor resultado foi a da imputação.